In [1]:
import requests

class BaseAPI:
    def __init__(self, base_url="https://4158-161-28-242-155.ngrok-free.app/v1"):
        self.base_url = base_url
        self.headers = {"Accept": "application/json"}  # Removed "Content-Type" for flexibility

    def _post(self, endpoint, data=None, files=None):
        url = f"{self.base_url}/{endpoint}"
        headers = self.headers
        if files:
            # For file uploads, do not include JSON data and let requests handle content-type
            response = requests.post(url, files=files, headers=headers)
        else:
            # For JSON data, specify json parameter
            response = requests.post(url, json=data, headers=headers)
        try:
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            print(f"HTTP Error: {e.response.status_code} {e.response.text}")
        except Exception as e:
            print(f"Unexpected Error: {str(e)}")
        return None

    def _get(self, endpoint):
        url = f"{self.base_url}/{endpoint}"
        response = requests.get(url, headers=self.headers)
        return response.json()

In [34]:

class ChatCompletionAPI(BaseAPI):
    def __init__(self, base_url="https://4158-161-28-242-155.ngrok-free.app/v1"):
        super().__init__(base_url)
        self.conversation_history = []
    
    def post_completions(self, messages, role = None, stream=True):
        """
        Post messages to the chat completions endpoint.
        
        :param messages: A list of message dictionaries with "content" and "role" keys.
        :param stream: Boolean to enable or disable streaming.
        """
        endpoint = "completions"
        if role:
            payload = {"prompt": messages, "role": role, "stream": stream}
        else:
            payload = {"prompt": messages, "stream": stream}
        response = self._post(endpoint, payload)
        return response

    def post_chat_completions(self, message, role = 'user', is_init = False, stream=False):
        endpoint = "chat/completions"
        if is_init:
            self.conversation_history = []  # Clear existing conversation history
            self.conversation_history.append({"content": message, "role": role})
            return

        # Add the new message with its role to the conversation history
        self.conversation_history.append({"content": message, "role": role})

        payload = {
            "messages": self.conversation_history#,
            #"stream": stream
        }
        response = self._post(endpoint, payload)  # And here
        return response


In [3]:
class IngestAPI(BaseAPI):
    def ingest_file(self, file_path):
        """Ingest a file to a specific endpoint."""
        endpoint = "ingest/file"
        with open(file_path, 'rb') as f:
            files = {'file': (file_path.split('/')[-1], f)}
            response = self._post(endpoint, files=files)
        return response

    def ingest_text(self, file_name, text):
        """Ingest text with a file name."""
        endpoint = "ingest/text"
        payload = {"file_name": file_name, "text": text}
        return self._post(endpoint, data=payload)

    def list_ingest_jobs(self):
        """List all ingest jobs."""
        endpoint = "ingest/list"
        return self._get(endpoint)

In [5]:
class UtilAPI(BaseAPI):
    def get_embeddings(self, input_text):
        """Post data to the embeddings endpoint."""
        endpoint = "embeddings"
        payload = {"input": input_text}
        return self._post(endpoint, data=payload)

    def check_health(self):
        """Get the health status of the API."""
        endpoint = "health"
        return self._get(endpoint)
    
    def post_chunks(self, text):
        """Post chunks of text."""
        endpoint = "chunks"
        payload = {"text": text}
        return self._post(endpoint, data=payload)

## Chatting

In [35]:
chat_api = ChatCompletionAPI()

In [36]:
chat_api = ChatCompletionAPI()

messages = "What is cs 2450 prerequisit?"

response = chat_api.post_completions(messages, stream=False)
print(response)


{'id': '24f50f47-09d4-429b-85fb-04eea80e04bf', 'object': 'completion', 'created': 1707737998, 'model': 'private-gpt', 'choices': [{'finish_reason': 'stop', 'delta': None, 'message': {'role': 'assistant', 'content': ' CS 2450, which is often referred to as "Data Structures and Algorithms II" or "Advanced Data Structures," is typically a graduate-level computer science course. The specific prerequisites for this course can vary depending on the university or college offering it. However, here are some common prerequisites:\n\n1. A strong foundation in programming concepts: Students should have a good understanding of programming fundamentals such as variables, control structures, functions, arrays, pointers, and data structures like linked lists and trees.\n2. Familiarity with a programming language: Students should be proficient in at least one programming language, such as C++, Java, Python, or C#.\n3. Basic knowledge of data structures: Students should have a good understanding of com

In [37]:
messages = "What is cs 2450 prerequisit?"
response = chat_api.post_completions(messages, role='user', stream=False)
print(response)

{'id': '75844b82-4d30-4b5a-875d-68564c37dba4', 'object': 'completion', 'created': 1707738009, 'model': 'private-gpt', 'choices': [{'finish_reason': 'stop', 'delta': None, 'message': {'role': 'assistant', 'content': ' CS 2450, which is often referred to as "Data Structures and Algorithms II" or "Advanced Data Structures," is typically a graduate-level computer science course. The specific prerequisites for this course can vary depending on the university or college offering it. However, here are some common prerequisites:\n\n1. A strong foundation in programming concepts: Students should have a good understanding of programming fundamentals such as variables, control structures (if statements, loops), functions, arrays, and pointers.\n2. Familiarity with a programming language: Students should be proficient in at least one programming language, such as C++, Java, Python, or C#.\n3. Basic knowledge of data structures: Students should have a solid understanding of basic data structures su

In [38]:
# Post chat completions with streaming enabled
stream_response = chat_api.post_chat_completions("You are UVU Advisor", role='system', stream=True, is_init=True)
stream_response = chat_api.post_chat_completions("What is cs 2450 prerequisit?", role='user', stream=True, is_init=False)
print("Stream Response:", stream_response)


Stream Response: {'id': 'f98d9002-77d5-4023-8ea0-6d156c5e5ed9', 'object': 'completion', 'created': 1707738018, 'model': 'private-gpt', 'choices': [{'finish_reason': 'stop', 'delta': None, 'message': {'role': 'assistant', 'content': ' CS 2450, which is "Data Structures and Algorithms II" at Utah Valley University (UVU), typically has the following prerequisites:\n\n1. CS 1510: Introduction to Computer Science I\n2. CS 1520: Introduction to Computer Science II\n3. CS 2410: Data Structures and Algorithms I\n\nIt\'s essential to note that students should consult the UVU catalog or speak with an academic advisor to confirm the most up-to-date prerequisites and any potential exceptions.\n\nCS 1510 and CS 1520 provide the fundamental programming concepts and skills needed for more advanced courses like CS 2450. CS 2410 focuses on data structures and algorithms, building a strong foundation for the advanced topics covered in CS 2450.\n\nIf you have any further questions or need clarification o

##  IngestAPI

In [10]:
file_paths = [
    ('/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf', 'application/octet-stream'),
    ('/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.txt', 'application/octet-stream')
]

files = [
    ('file', (file_path.split('/')[-1], open(file_path, 'rb'), content_type))
    for file_path, content_type in file_paths
]

response = requests.post(url, files=files)

# Correctly close the files after the request is made
for _, file_tuple in files:
    file_object = file_tuple[1]
    file_object.close()

print(response.text)

Making request to: https://4158-161-28-242-155.ngrok-free.app/v1/ingest/file with data: {'file': <_io.BufferedReader name='/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.json'>}
Unexpected Error: Object of type BufferedReader is not JSON serializable
File Ingest Response: None


In [ ]:
# Initialize the utility API client
util_api = UtilAPI()

# Get embeddings
embeddings_response = util_api.get_embeddings("This is a test sentence.")
print("Embeddings Response:", embeddings_response)

# Check health
health_response = util_api.check_health()
print("Health Response:", health_response)

# Post chunks of text
chunks_response = util_api.post_chunks("This is a test sentence for chunking.")
print("Chunks Response:", chunks_response)


### Totally tesitng


In [12]:

import requests

class GPTAPI:
    base_url = 'https://4158-161-28-242-155.ngrok-free.app/v1'

    def __init__(self, api_key):
        self.api_key = api_key

    def make_request(self, endpoint, data):
        headers = {
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json'
        }
        response = requests.post(f'{self.base_url}/{endpoint}', json=data, headers=headers)
        return response.json()


In [13]:
class CompletionAPI(GPTAPI):
    def __init__(self, api_key, streaming=False):
        super().__init__(api_key)
        self.endpoint = 'completions'
        self.streaming = streaming

    def complete(self, prompt, **kwargs):
        data = {'prompt': prompt, 'stream': self.streaming, **kwargs}
        return self.make_request(self.endpoint, data)

class ChatCompletionAPI(GPTAPI):
    def __init__(self, api_key, streaming=False):
        super().__init__(api_key)
        self.endpoint = 'chat/completions'
        self.streaming = streaming

    def chat(self, messages, **kwargs):
        data = {'messages': messages, 'stream': self.streaming, **kwargs}
        return self.make_request(self.endpoint, data)


In [15]:
api_key = 'your_api_key_here'

# Example: Non-streaming completion
completion_api = CompletionAPI(api_key, streaming=False)
response = completion_api.complete(prompt="Hello, world!", use_context=False)
print(response)




{'id': 'da36ddce-4b3b-4e71-b178-089f6f627c75', 'object': 'completion', 'created': 1707644164, 'model': 'private-gpt', 'choices': [{'finish_reason': 'stop', 'delta': None, 'message': {'role': 'assistant', 'content': ' Hello there! It\'s nice to meet you. How can I help you today? If you have any questions or topics you\'d like me to write about, feel free to ask. I\'m here to provide information and answer questions to the best of my ability. Let me know if you have any specific topic in mind or if you have any general questions. I\'ll do my best to provide accurate and helpful responses. Let me know if you have any preferences for length or format of the response as well. I look forward to helping you out!\n\nHere is a simple "Hello, World!" program in various programming languages for your reference:\n\n* C:\n```c\n#include <stdio.h>\n\nint main() {\n  printf("Hello, World!\\n");\n  return 0;\n}\n```\n\n* Java:\n```java\npublic class HelloWorld {\n  public static void main(String[] ar

In [16]:
# Example: Streaming chat completion
chat_api = ChatCompletionAPI(api_key, streaming=True)
response = chat_api.chat(messages=[{"role": "user", "content": "Hello!"}], use_context=False)
print(response)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [27]:
import requests

url = "https://4158-161-28-242-155.ngrok-free.app/v1/ingest/file"

# Replace these paths with the actual paths to your files
file_paths = [
    ('/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf', 'application/octet-stream'),
    ('/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.txt', 'application/octet-stream')
]

files = [
    ('file', (file_path.split('/')[-1], open(file_path, 'rb'), content_type))
    for file_path, content_type in file_paths
]

response = requests.post(url, files=files)

# Correctly close the files after the request is made
for _, file_tuple in files:
    file_object = file_tuple[1]
    file_object.close()

print(response.text)


{"object":"list","model":"private-gpt","data":[{"object":"ingest.document","doc_id":"1e236604-9b28-4e24-8b70-a9da271dd229","doc_metadata":{"file_name":"course_catelog_cs_uvu.txt"}}]}


In [33]:
import requests

class BaseAPI:
    def __init__(self, base_url="https://4158-161-28-242-155.ngrok-free.app/v1"):
        self.base_url = base_url
        self.headers = {"Accept": "application/json"}  # Removed "Content-Type" for flexibility

    def _post(self, endpoint, data=None, files=None):
        url = f"{self.base_url}/{endpoint}"
        headers = self.headers
        if files:
            # For file uploads, do not include JSON data and let requests handle content-type
            response = requests.post(url, files=files, headers=headers)
        else:
            # For JSON data, specify json parameter
            response = requests.post(url, json=data, headers=headers)
        try:
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            print(f"HTTP Error: {e.response.status_code} {e.response.text}")
        except Exception as e:
            print(f"Unexpected Error: {str(e)}")
        return None

    def _get(self, endpoint):
        url = f"{self.base_url}/{endpoint}"
        response = requests.get(url, headers=self.headers)
        return response.json()


In [ ]:
class IngestAPI(BaseAPI):
    def ingest_file(self, file_path):
        """Ingest a file to a specific endpoint."""
        endpoint = "ingest/file"
        with open(file_path, 'rb') as f:
            files = {'file': (file_path.split('/')[-1], f)}
            response = self._post(endpoint, files=files)
        return response

    def ingest_text(self, file_name, text):
        """Ingest text with a file name."""
        endpoint = "ingest/text"
        payload = {"file_name": file_name, "text": text}
        return self._post(endpoint, data=payload)

    def list_ingest_jobs(self):
        """List all ingest jobs."""
        endpoint = "ingest/list"
        return self._get(endpoint)


In [30]:
# Assuming the classes have been defined as per the previous instructions

# Initialize the API with the base URL
ingest_api = IngestAPI(base_url="https://4158-161-28-242-155.ngrok-free.app/v1")

# Example: Ingest a file
file_path = "/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf"
file_ingest_response = ingest_api.ingest_file(file_path)
print(file_ingest_response)

# Example: Ingest text
file_name = "course_catelog_cs_uvu.pdf"
text_content = "This is an example text content."
text_ingest_response = ingest_api.ingest_text(file_name, text_content)
print(text_ingest_response)

# Example: List ingest jobs
list_jobs_response = ingest_api.list_ingest_jobs()
print(list_jobs_response)


Making request to: https://4158-161-28-242-155.ngrok-free.app/v1/ingest/file
{'object': 'list', 'model': 'private-gpt', 'data': [{'object': 'ingest.document', 'doc_id': '7c29f12a-1bfe-43e7-b382-5febe79fe3cb', 'doc_metadata': {'page_label': '1', 'file_name': '/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf'}}, {'object': 'ingest.document', 'doc_id': 'd47ea662-4fa2-4a25-bf33-652533f4817f', 'doc_metadata': {'page_label': '2', 'file_name': '/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf'}}, {'object': 'ingest.document', 'doc_id': '576607e7-eafd-44f5-8391-81bae0d58c85', 'doc_metadata': {'page_label': '3', 'file_name': '/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf'}}, {'object': 'ingest.document', 'doc_id': '3c9db033-c3d9-4102-9a9a-76c7a33ec84e', 'doc_metadata': {'page_label': '4', 'file

AttributeError: 'IngestAPI' object has no attribute 'ingest_text'

In [32]:
class IngestAPI(BaseAPI):
    def ingest_file(self, file_path):
        """Ingest a file to a specific endpoint."""
        endpoint = "ingest/file"
        with open(file_path, 'rb') as f:
            files = {'file': (file_path.split('/')[-1], f)}
            response = self._post(endpoint, files=files)
        return response

    def ingest_text(self, file_name, text):
        """Ingest text with a file name."""
        endpoint = "ingest/text"
        payload = {"file_name": file_name, "text": text}
        return self._post(endpoint, data=payload)

    def list_ingest_jobs(self):
        """List all ingest jobs."""
        endpoint = "ingest/list"
        return self._get(endpoint)
    
    
ingest_api = IngestAPI(base_url="https://4158-161-28-242-155.ngrok-free.app/v1")   
file_name = "course_catelog_cs_uvu.pdf"
text_content = "This is an example text content."
text_ingest_response = ingest_api.ingest_text(file_name, text_content)
print(text_ingest_response)


Making request to: https://4158-161-28-242-155.ngrok-free.app/v1/ingest/text
HTTP Error: 500 Internal Server Error
None
